# Service - Flask - English Keyowrds & Sentiment

No individual Codes in Service Category - All are Flask Services for Flask Production Push - Inherently Optimized 

## Environemnt Versioning -- Need to Add 

## Github Push Check -- Last Updated Version of Notebook 

## Phase 0: Random Review Generator for Stress Test 

Using Amazon Review to create optimized service that can output random reviews. The API should expect N number to be generated. The backend storage of reviews should be limited to particular number and should be stored as flat files. 
 > Logic 
 
    - Store Million reviews in Flat files that can queried in millisecond ( Experiment & Research )

### Dump Review to Txt and then Gzip Compression

This is Batch - Need to run only once ; Can change the review category. Currently taking Kindle Review with ~1 Million review. 
As the million quantity increases, the batch caluclation size will also increase.

Here's link to other reviews:

Books : http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Books_5.json.gz   --> 8 Million

All Review:  http://jmcauley.ucsd.edu/data/amazon/

In [0]:
# This is a test. 
# This is a tesr

import pandas

In [0]:
#Libraries 
from flask import Flask, request
import pandas as pd
import numpy as np
import threading
import datetime
import socket
import gzip
import sys


print(socket.gethostbyname(socket.getfqdn(socket.gethostname())))

#Function 

def GenerateReviewCategoryLink(RawUserInput):
  """
  Raw User Input Mapping
  Music= "reviews_Musical_Instruments_5.json.gz" 
  
  ** Need to add otehr cateogries 
  """
  if RawUserInput == "Music":
    return "http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Musical_Instruments_5.json.gz"
  elif RawUserInput == "Kindle":
    return "http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Kindle_Store_5.json.gz"
    
def DumpJson2GZ(AmazonReviewLink, GzipLocation, TextInterLocation):
  # Give location with slash ending 
  data = pd.read_json(AmazonReviewLink, lines=True)
  GzipLocationFull = GzipLocation+'Reviews_compressed_'+"_"+datetime.datetime.today().strftime('%Y-%m-%d')+'.gz'
  TextInterLocationFull = TextInterLocation+'Reviews_text_'+"_"+datetime.datetime.today().strftime('%Y-%m-%d')+'.txt'
  np.savetxt(TextInterLocationFull, data['reviewText'].values,fmt='%s')
  input = open(TextInterLocationFull, 'rb')
  TextInputFromFile = input.read()
  output = gzip.GzipFile(GzipLocationFull, 'wb')
  output.write(TextInputFromFile)
  input.close()
  output.close()
  return "File Created"

app = Flask("ReviewCompressionApp_Batch")
@app.route('/ReviewDump')
def Json2GZService_Batch():
  AmazonReviewLink = "Kindle"
  GzipLocation = request.args.get('GzipLocation')
  TextInterLocation = request.args.get('TextInterLocation')
  return DumpJson2GZ(AmazonReviewLink, GzipLocation, TextInterLocation)
  
threading.Thread(target=app.run, kwargs={'host':'0.0.0.0','port':11}).start()
  

172.28.0.2
 * Serving Flask app "ReviewCompressionApp_Batch" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:11/ (Press CTRL+C to quit)


In [0]:
import requests
r = requests.get("http://172.28.0.2:11/ReviewDump?AmazonReviewLink=Music&GzipLocation=&TextInterLocation=")
print(r.status_code)
print(r.text)

172.28.0.2 - - [27/May/2019 06:49:52] "GET /ReviewDump?AmazonReviewLink=Music&GzipLocation=&TextInterLocation= HTTP/1.1" 200 -


200
File Created


In [0]:
!ps -fA | grep flask

root        2313    1719  0 11:20 ?        00:00:00 /bin/bash -c ps -fA | grep flask
root        2315    2313  0 11:20 ?        00:00:00 grep flask


In [0]:
!kill 2313

/bin/bash: line 0: kill: (2313) - No such process


### Generate random reviews from million review

In [0]:
#Libraries 
from flask import Flask, request
from itertools import islice
import pandas as pd
import numpy as np
import threading
import datetime
import random 
import socket
import gzip
import sys


print(socket.gethostbyname(socket.getfqdn(socket.gethostname())))

#Function 
def ReviewGenerator(CompressedFlag,CategoryOfReview,RandomIntensity,ReviewDumpLocation,NumberofReviews,DataType):
  """
  ** Need to add Category of Review
  """
  RandReview = []
  NumberofReviews = int(NumberofReviews)
  
#   if RandomIntensity=="Less":
#     RandomUpperLimit = 1000
#   elif RandomIntensity=="Medium":
#     RandomUpperLimit = 10000
#   elif RandomIntensity=="More":
#     RandomUpperLimit = 100000
#   else:  #Fastest, but less variation
#     RandomUpperLimit = 50
  RandomUpperLimit = RandomIntensity
  
  rangeValue = random.randint(1,RandomUpperLimit)
  
  if CompressedFlag:
    ReviewDumpLocationFull = ReviewDumpLocation+'Reviews_compressed_'+"_"+datetime.datetime.today().strftime('%Y-%m-%d')+'.gz'
    with gzip.open('text.gz','rt') as lines:
      for line in islice(lines, rangeValue, rangeValue+(NumberofReviews-1)):
        RandReview.append(line)
  
  else:
    ReviewDumpLocationFull = ReviewDumpLocation+'Reviews_text_'+"_"+datetime.datetime.today().strftime('%Y-%m-%d')+'.txt'
    with open(ReviewDumpLocationFull,'rt') as lines:
      for line in islice(lines, rangeValue, rangeValue+(NumberofReviews-1)):
        RandReview.append(line)
  
  return RandReview


app = Flask("GenerateRandomReviews")
@app.route('/ReviewGenerator')
def GenerateRandomReviews():
  """
  Compressed = ?
  Category (Kindle & Music) = 
  Random = Less, Medium,More  --> Less Random is fastest; More the random, more slow
  ReviewDump = Location
  NumberofReviews = Total Reviews 
  DataType = List
  """
  
  CompressedFlag = request.args.get('CompressFlag')
  CategoryOfReview = request.args.get("CategoryOfReview")
#   RandomIntensity = request.args.get("RandomIntesnity")
  RandomIntensity = request.args.get("RandomVariability")
  ReviewDumpLocation = request.args.get("ReviewDumpLocation")
  NumberofReviews = request.args.get("NumberofReviews")
  DataType = "List"   # Defaul List of Strings   
#   return ",".join(ReviewGenerator(CompressedFlag,CategoryOfReview,RandomIntensity,ReviewDumpLocation,NumberofReviews,DataType))
  return ReviewGenerator(CompressedFlag,CategoryOfReview,RandomIntensity,ReviewDumpLocation,NumberofReviews,DataType)
threading.Thread(target=app.run, kwargs={'host':'0.0.0.0','port':14}).start()

172.28.0.2
 * Serving Flask app "GenerateRandomReviews" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:14/ (Press CTRL+C to quit)


In [0]:
import requests
r = requests.get("http://172.28.0.2:14/ReviewGenerator?CompressedFlag=False&RandomIntensity=More&ReviewDumpLocation=&NumberofReviews=1")
print(r.status_code)
print(r.text)

172.28.0.2 - - [27/May/2019 10:08:19] "GET /ReviewGenerator?CompressedFlag=False&RandomIntensity=More&ReviewDumpLocation=&NumberofReviews=1 HTTP/1.1" 200 -


200



## Phase 1: Extarct & Dump Text Feature - Row Wise

Pickling is not working. Currently saving CSV file. 

In [0]:
# Install Libraries
import spacy
spacy.cli.download("en_core_web_md")
!pip install rake_nltk
!pip install textacy


    Linking successful
    /usr/local/lib/python3.6/dist-packages/en_core_web_md -->
    /usr/local/lib/python3.6/dist-packages/spacy/data/en_core_web_md

    You can now load the model via spacy.load('en_core_web_md')

  Stored in directory: /root/.cache/pip/wheels/ef/92/fc/271b3709e71a96ffe934b27818946b795ac6b9b8ff8682483f
Successfully built rake-nltk
     |████████████████████████████████| 163kB 2.8MB/s 
     |████████████████████████████████| 184kB 37.6MB/s 
     |████████████████████████████████| 3.0MB 42.4MB/s 
     |████████████████████████████████| 51kB 19.8MB/s 
     |████████████████████████████████| 5.4MB 35.2MB/s 
  Stored in directory: /root/.cache/pip/wheels/de/c2/93/660fd5f7559049268ad2dc6d81c4e39e9e36518766eaf7e342
Successfully built python-levenshtein
  Found existing installation: scikit-learn 0.21.2
    Uninstalling scikit-learn-0.21.2:
      Successfully uninstalled scikit-learn-0.21.2


In [0]:
#Libraries 
from flask import Flask, request
from itertools import islice
import pandas as pd
import numpy as np
import threading
import dill as pickle
import datetime
import random 
import socket
import json
import gzip
import sys
import spacy
import tqdm
import time 
import pandas as pd
from textblob import TextBlob
from rake_nltk import Metric, Rake
from textblob.np_extractors import ConllExtractor
extractor = ConllExtractor()
import textacy
import nltk
from nltk.corpus import stopwords 
from dask import dataframe as dd
import textacy.keyterms
import pickle



print(socket.gethostbyname(socket.getfqdn(socket.gethostname())))

def getFeature(row):
    FeatureDict = {}
    # Rake Topic 
    r = Rake(min_length=2, max_length=4) # Uses stopwords for english from NLTK, and all puntuation characters.
    r.extract_keywords_from_text(row)
    topics = r.get_ranked_phrases()
    
    text = nlp(row)
    # Noun Phrase 
    
    noun_phrase_pairs = []
    for chunk in text.noun_chunks:
    #   print (chunk.text)
      if len(chunk.text.split(" "))>1 and chunk.text not in nltk.corpus.stopwords.words('english'):
        noun_phrase_pairs.append(chunk.text)
        
    # Sentiment 
    Sentiment = float("{0:.2f}".format(TextBlob(row).sentiment.polarity))
    
    # Negation Pairs 
    Negation_Pair = []
    negation_tokens = [tok for tok in text if tok.dep_ == 'neg']
    negation_head_tokens = [token.head for token in negation_tokens]
    for token in negation_head_tokens:
      Negation_Pair.append([token.text, token.dep_, token.head.text, token.head.pos_, [child for child in token.children]])
    
    # NMF Topics 
      ## Not Now
      
    # Basic Entity Extraction 
      ## Not now 
    
    # Noun Adjective Pair ---> Rishi 
    output = []
    a_n_pair = []
    
    n_a_pair = []
    for sent in text.sents: # sentences of a doc
      # This block handles adjective noun pairs
      for token in sent: # tokens of a sentence
        if token.pos_ == "VERB" : # since verbs are central connection between nouns and adjs
#           print(token, "is a verb")
            for child in token.children: # childen of verbs
#             print('child ', child)
              if child.pos_ in ['NOUN', 'PRON','PROPN']: 
                for grand_child in child.children:
#                 print("grandchild", grand_child)
                  if grand_child.pos_ in ['ADJ']:
                    a_n_pair.append(grand_child.string.strip()+" "+child.string.strip())
      if len(a_n_pair)>0:
        output.extend(a_n_pair)
      
      # This block handles noun adj pairs
      noun_flag = False
      noun = str
      adj = str()
      adj_flag = False
      for token in sent:
        if token.pos_ == "VERB" :
#           print(token, "is a verb")
            for child in token.children:
#             print('child2 ', child)
              if noun_flag == False:
                if child.pos_ in ['NOUN', 'PRON','PROPN']:
                  noun_flag = True
                  noun = child
                  continue
              if adj_flag == False:
                if child.pos_ in ['ADJ']:
                  adj_flag = True
                  adj = child
              if noun_flag and adj_flag:
                n_a_pair.append(adj.string.strip()+" "+noun.string.strip())

      if len(n_a_pair)>0:
        output.extend(n_a_pair)
    output = list(set(output))
    
    # Subject Verb Object 
    SVOPair = list(textacy.extract.subject_verb_object_triples(text))
    # SG Rank 
    SGRankTopic = list(textacy.keyterms.sgrank(text, ngrams=(1, 2, 3, 4, 5, 6), normalize='lemma', window_width=1500, n_keyterms=10, idf=None))
    # TextRank 
    TextRank = list(textacy.keyterms.textrank(text, normalize='lemma', n_keyterms=10))
    # Key Terms Semantic Network 
    KeyTermPageRank = list(textacy.keyterms.key_terms_from_semantic_network(text, normalize='lemma', window_width=2, edge_weighting='binary', ranking_algo='pagerank', join_key_words=False, n_keyterms=5))
    
    # Topic from NMF 
    FeatureDict['RakeTopics'] = topics
    FeatureDict['NounPhrase'] = noun_phrase_pairs
    FeatureDict['Sentiment'] = Sentiment
    FeatureDict['Negation_Pair'] = Negation_Pair
    FeatureDict['SubjectVerbObject'] = SVOPair
    FeatureDict['SGRankTopic'] = SGRankTopic
    FeatureDict['TextRank'] = TextRank
    FeatureDict['PageRankTopic'] = KeyTermPageRank
    FeatureDict['NounAdjectivePair'] = output
    FeatureDict['SPacyDocObject'] = [nlp(row)]
    return FeatureDict
  
def getDocDump(row,nlp):
  NLPObject = {}
  NLPObject['Object'] = [nlp(row)]
  return NLPObject


app = Flask("GenerateRandomReviews")
@app.route('/TextFeatureDump')
def ExtractReviews():
  """
  Takes DataFrame --> NPZ to DF 
  
  """
#   DF = request.args.get('DFLocation')
  
  global nlp
  nlp = spacy.load("en_core_web_md")
#   nlp = spacy.load("en")
  data = pd.read_json("http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Beauty_5.json.gz", lines=True)

  data = data.head(10)
  ddata = dd.from_pandas(data, npartitions=4)
#   %time ddata['DocObject']  = ddata.map_partitions(lambda df: df.reviewText.apply((lambda row: getDocDump(row,nlp)))).compute(scheduler='threads')
  %time ddata['ReviewFeature'] = ddata.map_partitions(lambda df: df.reviewText.apply((lambda row: getFeature(row)))).compute(scheduler='synchronous', optimize_graph='True')
  df = ddata.compute()
  
#   df.to_pickle("TestPickle.pickle")
  df.to_csv("TestAPI.csv")
#   with open("backup", 'wb') as f:
#     pickle.dump(df, f)
#   df.to_pickle("Test_Pickle.pickle")
#   dfjson = df.to_json()
#   with open('data.txt', 'w') as outfile:  
#     json.dumps(dfjson)
#   outfile.close()
  
  
#   return (df.to_json(orient='records'))
  return ("DOne")




threading.Thread(target=app.run, kwargs={'host':'0.0.0.0','port':47}).start()


172.28.0.2
 * Serving Flask app "GenerateRandomReviews" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:47/ (Press CTRL+C to quit)


In [0]:
import requests
%time r = requests.get("http://172.28.0.2:47/TextFeatureDump")
print(r.status_cosde)
# print(r.text[0:10])

CPU times: user 28min 20s, sys: 22min 13s, total: 50min 34s
Wall time: 25min 49s


172.28.0.2 - - [03/Jun/2019 11:13:47] "GET /TextFeatureDump HTTP/1.1" 200 -


CPU times: user 28min 47s, sys: 22min 14s, total: 51min 1s
Wall time: 26min 22s
200


## Phase 2: Extract & Dump Text Features - Whole Review -> Topic 

## Phase 3: Whole Review -> Topic -> Mapping to Rows 

## Phase 4: Best Topic Calculation Logic 

# Optimization Experiments - MutiThread 

## Review -> Txt -> Gzip

In [0]:
import pandas as pd
data = pd.read_json("http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Kindle_Store_5.json.gz", lines=True)

In [0]:
import numpy as np 
np.savetxt(r'np.txt', data['reviewText'].values,fmt='%s')

In [0]:
import zlib, base64
file = open('np.txt','r')
text = file.read()
file.close()
code =  base64.b64encode(zlib.compress(text.encode('utf-8'),9))
code = code.decode('utf-8')
# print(code)

In [0]:
import gzip
input = open("np.txt", 'rb')
s = input.read()
input.close()

output = gzip.GzipFile("text.gz", 'wb')
output.write(s)
output.close()

In [0]:
# import gzip
# import random
# import mmap

# with gzip.open('text.gz','rt') as f:
#   for lines in range(500):
#     line = f.readline()
# #     lines = random.sample(f.readlines(),5)
# #     print(line)
#     print(random.sample(line,5))

# for lines in range(56):
#   if lines==33:
#     line = gzip.open('text.gz','rt').readline()
#   #     lines = random.sample(f.readlines(),5)
#   #     print(line)
#     print(line)
#   print(random.sample(line,1))

from itertools import islice
import random 
rangeValue = random.randint(1,10000)
TotalReview = 5
# print(rangeValue)
with gzip.open('text.gz','rt') as lines:
    for line in islice(lines, rangeValue, rangeValue+(TotalReview-1)):
        print (line)


# with gzip.open('text.gz','rt') as f:
#   for lines in range(500):
#   mm = mmap.mmap(f.fileno(), 0)
#   print(random.sample(mm.readlines(),5))
#   print(mm.readline()) 
#   line = next(f)
#   for num, aline in enumerate(f, 2):
#     if random.randrange(num): continue
#     line = aline
#   print(line)

# def random_line(afile):
#     line = next(afile)
#     for num, aline in enumerate(afile, 2):
#       if random.randrange(num): continue
#       line = aline
#     return line

The main character, Two, is a nineteen year-old prostitute, hooked on heroin. She is trapped in her life and has abanboned all hopes of life getting any better.Her pimp sends her to a new client and she mets Theroen, a powerful vampire who starts to change her.Due to the drugs in her system, Two does not become a vampire right away and remains a few months with Theroen before the final change. During that time, she discovers how intricate and dangerous the politics of vampire life are.This is not your typical vampire story. I  kept being pleasantly surprised by new concepts. The plot is well spaced and kept me interested until the end and, as other readers, i am willing to read the sequel.I gave it four stars instead of five because I kept wishing that the author explain, and better showcase, the love and connection between Theroen and Two. We get a very simple and quick explanation of why Theroen chose Two but it didn't feel real, logical or emotional to me. Also, the vampire scene, l

## Loading Object with parallelization

In [0]:
import spacy
spacy.cli.download("en_core_web_md")
!pip install rake_nltk
!pip install textacy


    Linking successful
    /usr/local/lib/python3.6/dist-packages/en_core_web_md -->
    /usr/local/lib/python3.6/dist-packages/spacy/data/en_core_web_md

    You can now load the model via spacy.load('en_core_web_md')

  Stored in directory: /root/.cache/pip/wheels/ef/92/fc/271b3709e71a96ffe934b27818946b795ac6b9b8ff8682483f
Successfully built rake-nltk
     |████████████████████████████████| 163kB 4.9MB/s 
     |████████████████████████████████| 51kB 17.4MB/s 
     |████████████████████████████████| 286kB 46.4MB/s 
     |████████████████████████████████| 3.0MB 45.8MB/s 
     |████████████████████████████████| 5.4MB 36.4MB/s 
  Stored in directory: /root/.cache/pip/wheels/de/c2/93/660fd5f7559049268ad2dc6d81c4e39e9e36518766eaf7e342
Successfully built python-levenshtein
ERROR: pymc3 3.6 has requirement joblib<0.13.0, but you'll have joblib 0.13.2 which is incompatible.
  Found existing installation: joblib 0.12.5
    Uninstalling joblib-0.12.5:
      Successfully uninstalled joblib-0.12

In [0]:
# Current Benchmark with 10,000 reiews on 4 core machine and 8 GB - Takes 8 min 

from dask import dataframe as dd
import spacy
import tqdm
import time 
import pandas as pd
from textblob import TextBlob
from rake_nltk import Metric, Rake
from textblob.np_extractors import ConllExtractor
extractor = ConllExtractor()
import textacy
import nltk
from nltk.corpus import stopwords 
import textacy.keyterms
data = pd.read_json("http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Musical_Instruments_5.json.gz", lines=True)

data = data.head(10)
ddata = dd.from_pandas(data, npartitions=4)

def getFeature(row):
    # Rake Topic 
    r = Rake(min_length=2, max_length=4) # Uses stopwords for english from NLTK, and all puntuation characters.
    r.extract_keywords_from_text(row)
    topics = r.get_ranked_phrases()
    
    text = nlp(row)
    # Noun Phrase 
    
    noun_phrase_pairs = []
    for chunk in text.noun_chunks:
    #   print (chunk.text)
      if len(chunk.text.split(" "))>1 and chunk.text not in nltk.corpus.stopwords.words('english'):
        noun_phrase_pairs.append(chunk.text)
        
    # Sentiment 
    Sentiment = float("{0:.2f}".format(TextBlob(row).sentiment.polarity))
    
    # Negation Pairs 
    Negation_Pair = []
    negation_tokens = [tok for tok in text if tok.dep_ == 'neg']
    negation_head_tokens = [token.head for token in negation_tokens]
    for token in negation_head_tokens:
      Negation_Pair.append([token.text, token.dep_, token.head.text, token.head.pos_, [child for child in token.children]])
    
    # NMF Topics 
      ## Not Now
      
    # Basic Entity Extraction 
      ## Not now 
    
    # Noun Adjective Pair ---> Rishi 
    output = []
    a_n_pair = []
    
    n_a_pair = []
    for sent in text.sents: # sentences of a doc
      # This block handles adjective noun pairs
      for token in sent: # tokens of a sentence
        if token.pos_ == "VERB" : # since verbs are central connection between nouns and adjs
#           print(token, "is a verb")
            for child in token.children: # childen of verbs
#             print('child ', child)
              if child.pos_ in ['NOUN', 'PRON','PROPN']: 
                for grand_child in child.children:
#                 print("grandchild", grand_child)
                  if grand_child.pos_ in ['ADJ']:
                    a_n_pair.append(grand_child.string.strip()+" "+child.string.strip())
      if len(a_n_pair)>0:
        output.extend(a_n_pair)
      
      # This block handles noun adj pairs
      noun_flag = False
      noun = str
      adj = str()
      adj_flag = False
      for token in sent:
        if token.pos_ == "VERB" :
#           print(token, "is a verb")
            for child in token.children:
#             print('child2 ', child)
              if noun_flag == False:
                if child.pos_ in ['NOUN', 'PRON','PROPN']:
                  noun_flag = True
                  noun = child
                  continue
              if adj_flag == False:
                if child.pos_ in ['ADJ']:
                  adj_flag = True
                  adj = child
              if noun_flag and adj_flag:
                n_a_pair.append(adj.string.strip()+" "+noun.string.strip())

      if len(n_a_pair)>0:
        output.extend(n_a_pair)
    output = list(set(output))
    
    # Subject Verb Object 
    SVOPair = list(textacy.extract.subject_verb_object_triples(text))
    # SG Rank 
    SGRankTopic = list(textacy.keyterms.sgrank(text, ngrams=(1, 2, 3, 4, 5, 6), normalize='lemma', window_width=1500, n_keyterms=10, idf=None))
    # TextRank 
    TextRank = list(textacy.keyterms.textrank(text, normalize='lemma', n_keyterms=10))
    # Key Terms Semantic Network 
    KeyTermPageRank = list(textacy.keyterms.key_terms_from_semantic_network(text, normalize='lemma', window_width=2, edge_weighting='binary', ranking_algo='pagerank', join_key_words=False, n_keyterms=5))
    
    # Topic from NMF 
    
    return (topics,noun_phrase_pairs, Sentiment, Negation_Pair, SVOPair, SGRankTopic, TextRank, KeyTermPageRank,output )
  
  
def getDocDump(row):
  return [nlp(row)]

global nlp
nlp = spacy.load("en_core_web_md")
%time ddata['DocObject']  = ddata.map_partitions(lambda df: df.reviewText.apply((lambda row: getDocDump(row)))).compute(scheduler='threads')
%time ddata['ReviewFeature'] = ddata.map_partitions(lambda df: df.reviewText.apply((lambda row: getFeature(row)))).compute(scheduler='threads')
df = ddata.compute()
# df.to_csv("testoutput.csv")

CPU times: user 37.4 s, sys: 1.7 s, total: 39.1 s
Wall time: 38.5 s
CPU times: user 23.9 s, sys: 1.61 s, total: 25.5 s
Wall time: 24.8 s


In [0]:
df.head(10s)

,asin,helpful,overall,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime,DocObject,ReviewFeature
0,1384719342,"[0, 0]",5,"Not much to write about here, but it does exac...","02 28, 2014",A2IBPI20UZIR0U,"cassandra tu ""Yeah, well, that's just like, u...",good,1393545600,"[(Not, much, to, write, about, here, ,, but, i...","([lowest prices pop filters, pop sounds, well ..."
1,1384719342,"[13, 14]",5,The product does exactly as it should and is q...,"03 16, 2013",A14VAT5EAX3D9S,Jake,Jake,1363392000,"[(The, product, does, exactly, as, it, should,...","([pleasing aroma like mine, old grape candy, c..."
2,1384719342,"[1, 1]",5,The primary job of this device is to block the...,"08 28, 2013",A195EZSQDW3E21,"Rick Bennette ""Rick Bennette""",It Does The Job Well,1377648000,"[(The, primary, job, of, this, device, is, to,...","([mike stand secure enough, metal clamp mount ..."
3,1384719342,"[0, 0]",5,Nice windscreen protects my MXL mic and preven...,"02 14, 2014",A2C00NNG1ZQQG2,"RustyBill ""Sunday Rocker""",GOOD WINDSCREEN FOR THE MONEY,1392336000,"[(Nice, windscreen, protects, my, MXL, mic, an...","([requires careful positioning, nice windscree..."
4,1384719342,"[0, 0]",5,This pop filter is great. It looks and perform...,"02 21, 2014",A94QU4C90B1AX,SEAN MASLANKA,No more pops when I record my vocals.,1392940800,"[(This, pop, filter, is, great, ., It, looks, ...","([studio filter, recording vocals, pop filter,..."
5,B00004Y2UT,"[0, 0]",5,So good that I bought another one. Love the h...,"12 21, 2012",A2A039TZMZHH9Y,"Bill Lewey ""blewey""",The Best Cable,1356048000,"[(So, good, that, I, bought, another, one, ., ...","([still works great, learned last night, bough..."
6,B00004Y2UT,"[0, 0]",5,"I have used monster cables for years, and with...","01 19, 2014",A1UPZM995ZAH90,Brian,Monster Standard 100 - 21' Instrument Cable,1390089600,"[(I, have, used, monster, cables, for, years, ...","([used monster cables, cables break, simple fa..."
7,B00004Y2UT,"[0, 0]",3,I now use this cable to run from the output of...,"11 16, 2012",AJNFQI3YR6XJ5,"Fender Guy ""Rick""",Didn't fit my 1996 Fender Strat...,1353024000,"[(I, now, use, this, cable, to, run, from, the...","([would try another one, planet waves cable ....."
8,B00004Y2UT,"[0, 0]",5,Perfect for my Epiphone Sheraton II. Monster ...,"07 6, 2008",A3M1PLEYNDEYO8,"G. Thomas ""Tom""",Great cable,1215302400,"[(Perfect, for, my, Epiphone, Sheraton, II, .,...","([epiphone sheraton ii, 90 degree plug, well c..."
9,B00004Y2UT,"[0, 0]",5,Monster makes the best cables and a lifetime w...,"01 8, 2014",AMNTZU1YQN1TH,Kurt Robair,Best Instrument Cables On The Market,1389139200,"[(Monster, makes, the, best, cables, and, a, l...","([cant go wrong, bass guitar rig, works great,..."


##Doing Feature Extraction on Whole Review as One

In [0]:
# Current Benchmark with 10,000 reiews on 4 core machine and 8 GB - Takes 8 min 

from dask import dataframe as dd
import spacy
import tqdm
import time 
import pandas as pd
from textblob import TextBlob
from rake_nltk import Metric, Rake
from textblob.np_extractors import ConllExtractor
extractor = ConllExtractor()
import textacy
import nltk
from nltk.corpus import stopwords 
import textacy.keyterms
data = pd.read_json("http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Beauty_5.json.gz", lines=True)

data = data.head(1000)
ddata = dd.from_pandas(data, npartitions=4)

In [0]:
def clean_text(df):
    '''
    Take in a Dataframe, and process it
    '''
#     df["cleaned_text"] = df.text.map(lambda text:text.lower()).map(remove_emails).map(remove_newlinechars).map(remove_stopwords).map(lemmatize)
    text = ",".join(df['reviewText'].values)
    r = Rake(min_length=2, max_length=4)
    r.extract_keywords_from_text(text)
    topics = [r.get_ranked_phrases()]
    return topics
  
def clean_text2(df):
    text = ",".join(df['reviewText'].values)
    text = nlp(text)
    noun_phrase_pairs = []
    for chunk in text.noun_chunks:
#   print (chunk.text)
      if len(chunk.text.split(" "))>1 and chunk.text not in nltk.corpus.stopwords.words('english'):
        noun_phrase_pairs.append(chunk.text)
    return noun_phrase_pairs
  
def clean_text3(df):
    text = ",".join(df['reviewText'].values)
    text = nlp(text)
    return list(textacy.extract.subject_verb_object_triples(text))
  
def clean_text4(df):
    text = ",".join(df['reviewText'].values)
    text = nlp(text)
    return list(textacy.keyterms.sgrank(text, ngrams=(2, 3, 4), normalize='lemma', window_width=1500, n_keyterms=10, idf=None))

def clean_text5(df):
    text = ",".join(df['reviewText'].values)
    text = nlp(text)
    return list(textacy.keyterms.textrank(text,normalize='lemma', n_keyterms=10))
  
  
global nlp
nlp = spacy.load("en_core_web_md")

result = ddata.map_partitions(clean_text3, meta=data)
%time df = result.compute(scheduler='threads')
df


CPU times: user 19.3 s, sys: 3.05 s, total: 22.4 s
Wall time: 11.6 s


0    [((palette), (was), (price)), ((it), (has), (c...
1    [((It), (gets), (range)), ((I), ('ve, used), (...
2    [((I), (give), (product)), ((There), (are), (t...
3    [((I), (prefer), (sponges)), ((I), (prefer), (...
dtype: object

In [0]:
text = ",".join(data['reviewText'].values)
text = nlp(text)
%time list(textacy.extract.subject_verb_object_triples(text))

CPU times: user 272 ms, sys: 0 ns, total: 272 ms
Wall time: 272 ms


[(palette, was, price),
 (it, has, coverage),
 (it, has, variety),
 (I, use, two),
 (which, is, reason),
 (I, 'm giving, it),
 (I, recommend, it),
 (It, 's not, powder),
 (It, 's not, kind),
 (it, show, color),
 (it, lasted, me),
 (it, does, coverage),
 (It, dries, skin),
 (I, 'd recommend, it),
 (you, can do, lot),
 (I, have, normal),
 (I, have, to dry),
 (I, 'm, years),
 (I, use, primer),
 (You, get, amount),
 (DONT, DO, IT),
 (makeup, to stay, put),
 (I, put, it),
 (I, used, tube),
 (it, did, nothing),
 (it, did, Lightening),
 (it, did, Brightening,),
 (it, tends, to dry),
 (it, tends, made),
 (didn't.,I, bought, product),
 (it, did help, bit),
 (I, saw, lot),
 (it, would have worked, better.,I),
 (I, started, using),
 (It, started, peel),
 (There, 's, bit),
 (I, 've stopped, using),
 (I, reached, skin color),
 (it, has, expiration date),
 (it, has, something),
 (it, worked, for),
 (it, bought, product),
 (didn't.,I, bought, product),
 (I, finished, tube),
 (I, started, seeing),
 (I

In [0]:
df[0]

[(palette, was, price),
 (it, has, coverage),
 (it, has, variety),
 (I, use, two),
 (which, is, reason),
 (I, 'm giving, it),
 (I, recommend, it),
 (It, 's not, powder),
 (It, 's not, kind),
 (it, show, color),
 (it, lasted, me),
 (it, does, coverage),
 (It, dries, skin),
 (I, 'd recommend, it),
 (you, can do, lot),
 (I, have, normal),
 (I, have, to dry),
 (I, 'm, years),
 (I, use, primer),
 (You, get, amount),
 (DONT, DO, IT),
 (makeup, to stay, put),
 (I, put, it),
 (I, used, tube),
 (it, did, nothing),
 (it, did, Lightening),
 (it, did, Brightening,),
 (it, tends, to dry),
 (it, tends, made),
 (didn't.,I, bought, product),
 (it, did help, bit),
 (I, saw, lot),
 (it, would have worked, better.,I),
 (I, started, using),
 (It, started, peel),
 (There, 's, bit),
 (I, 've stopped, using),
 (I, reached, skin color),
 (it, has, expiration date),
 (it, has, something),
 (it, worked, for),
 (it, bought, product),
 (didn't.,I, bought, product),
 (I, finished, tube),
 (I, started, seeing),
 (I

In [0]:
df[1]

[(It, gets, range),
 (I, 've used, it),
 (clamp, holds, hair),
 (wire foot, keep, barrel),
 (plastic tip, keep, barrel),
 (It, has, button),
 (it, 's left, untouched),
 (I, use, modelduring),
 (I, 've had, haircut),
 (they, 're, irons),
 (I, love, heat feature),
 (who, tend, to forget),
 (they, left, iron),
 (I, 'm not, one),
 (I, appreciate, feature),
 (settings, are, feature),
 (well!,I, ordered, two),
 (I, have, hair),
 (I, use, this),
 (youtube, has, videos),
 (I, release, curl),
 (It, takes, iron works),
 (it, fills, space),
 (I, like, dial design),
 (you, plan, to move),
 (Iron, curled, hair),
 (I, got, it),
 (It, doesn't take, heat),
 ('s, is, heat settings),
 (I, hold, it),
 (I, hold, it),
 (it, lasted, me),
 (You, cannot beat, that.,I),
 (I, ordered, it),
 (I, used, it),
 (it, kept, turning),
 (it, kept, shorting),
 (I, changed, outlets),
 (I, would recommend, iron),
 (it, was, one),
 (it, was, time),
 (it, heat, enough),
 (it, heat, to hold),
 (it, 's, product),
 (I, will not

In [0]:
df[3]

[(I, prefer, sponges),
 (I, prefer, scrubs),
 (I, love, Pufs),
 (They, are, exfoliation tool),
 (It, 's, spring cleaning),
 (It, was, to see),
 (I, would visit, area),
 (there, were, deer),
 (I, would see, ticks),
 (I, rinse, them),
 (it, fits, Puf),
 (it, fits, scrub handle),
 (I, kept, it),
 (Amazon, seems, have),
 (place, seems, have),
 (body sponge, has, side),
 (necessary.,I, have used, product),
 (It, preps, skin),
 (I, use, Puf handle),
 (exfoliate, purchased, this),
 (what, help, prevent),
 (I, use, dab),
 (It, is, exfoliation),
 (It, is, to get),
 (it, irritates, skin),
 (puff, has helped, make),
 (I, have used, one),
 (I, have gotten, two),
 (I, have gotten, weeks),
 (it, was beginning, to lose),
 (I, had had, one),
 (it, does, job),
 (exfoliating!,I, bought, sponge),
 (I, ended, using),
 (it, was doing, anything),
 (I, ended, scrubbing),
 (it, is, gentle&#34),
 (I, purchased, intensity.,Ok),
 (I, wanted, buffer),
 (I, could use, it),
 (I, use, it),
 (air, dry, it),
 (It, doe

In [0]:
df[2]

[('-PRON- hair', 0.2584553075012388),
 ('-PRON- scalp', 0.15023490286612437),
 ('-PRON- skin', 0.08085802279217426),
 ('-PRON- face', 0.07394901093610207),
 ('coal tar', 0.0360102538466297),
 ('-PRON- hand', 0.028851298954865733),
 ('product that', 0.02853735723872345),
 ('curly hair', 0.02825706076198932),
 ('-PRON- foot', 0.027350905223094038),
 ('little bit', 0.027109141519387814)]

In [0]:
text = ",".join(data['reviewText'].values)
r = Rake(min_length=2, max_length=4)
r.extract_keywords_from_text(text)
topics = r.get_ranked_phrases_with_scores()

In [0]:
topics

[(15.0, 'metal clamp mount attaches'),
 (15.0, 'candy apple red tele'),
 (14.846153846153847, 'roc n soc ...'),
 (14.785714285714285, 'medium sized venue .,'),
 (14.75, 'lowest prices pop filters'),
 (14.666666666666666, 'mailer inside another box'),
 (14.625, 'pleasing aroma like mine'),
 (14.333333333333334, 'short heat shrink overwrap'),
 (14.0, 'shoe mounted shotgun mike'),
 (13.833333333333334, 'mike stand secure enough'),
 (13.785714285714285, '., nice windscreen protects'),
 (13.75, 'double cloth filter blocks'),
 (13.75, 'cable meets connector ),'),
 (13.666666666666666, 'pretty good sweet spot'),
 (13.666666666666666, 'behringer xenyx 302 mixer'),
 (13.6, 'molded stress relief ends'),
 (13.589285714285715, 'high quality silicone jackets'),
 (13.416666666666666, 'recieved one bad cable'),
 (13.416666666666666, 'necessarily equal higher value'),
 (13.25, 'thick rubber cover !!'),
 (13.2, 'xlr 3 pin adaptor'),
 (13.059523809523808, 'cool looking cheap cable'),
 (12.92857142857142

In [0]:
global nlp
nlp = spacy.load("en_core_web_md")
text = nlp(text)
noun_phrase_pairs = []
for chunk in text.noun_chunks:
#   print (chunk.text)
  if len(chunk.text.split(" "))>1 and chunk.text not in nltk.corpus.stopwords.words('english'):
    noun_phrase_pairs.append(chunk.text)

ValueError: ignored

In [0]:
df[3]

[['tweeters sport silk domes',
  'mandobanjoacoustic guitarupright bassreg bassdon',
  'singular without risking ostracization',
  'blah blah blah ...',
  'widely compatible --&# 62',
  'monte allums sd808 mod',
  'dc18v 100mavisual sound onespot',
  'andy summers police sound',
  'blackand straplocksschaller security locks',
  'making violent purple flashes',
  'audioengine d1 dac connected',
  '034 /. 036 respectively',
  'couplers &# 34 ;.',
  'audiophile &# 34 ;.',
  'les paul signature stamps',
  '&# 60 ;$ 125',
  'far costlier sony pcm',
  'novelty &# 34 ;,',
  "brookmays ', ignorant responses",
  'swing trees &# 34',
  'turds .,*** update ***',
  'epiphone les paul wore',
  'epiphone les paul specail',
  '320 kbps mp3 recordings',
  'mans health really reccomend',
  'pics &# 34 ;.',
  'old motu 828 mkii',
  'humm ., apple threw',
  '1990s trace acoustic tap',
  'beforegood pricemetronome flasher maybe',
  'trippy modern hifidelity effects',
  'shiny runner like materail',
  'inf

In [0]:
def testf(df, col1, RakeObject):
#   text = ".".join(ddata[col1].values)
#   print(ddata[col1].values)
#   print(text)
#   # Rake Topic 
#   r = Rake(min_length=2, max_length=4)
#   r.extract_keywords_from_text(text)
#   topics = r.get_ranked_phrases()
  
#   # Subject Verb Object 
# #   SVOPair = list(textacy.extract.subject_verb_object_triples(text)) 
#   return (topics)
#     print("".join(df[col1].values))
    RakeObject.extract_keywords_from_text(",".join(df[col1].values))
#     print(RakeObject.get_ranked_phrases())
    return df.assign(result = [RakeObject.get_ranked_phrases()])
  
global nlp
nlp = spacy.load("en_core_web_md")
r = Rake(min_length=2, max_length=4)
dff = ddata.map_partitions(testf, 'reviewText', r).compute(scheduler='multiprocess')

ValueError: ignored

In [0]:
dff

array(["Not much to write about here, but it does exactly what it's supposed to. filters out the pop sounds. now my recordings are much more crisp. it is one of the lowest prices pop filters on amazon so might as well buy it, they honestly work the same despite their pricing,",
       "The product does exactly as it should and is quite affordable.I did not realized it was double screened until it arrived, so it was even better than I had expected.As an added bonus, one of the screens carries a small hint of the smell of an old grape candy I used to buy, so for reminiscent's sake, I cannot stop putting the pop filter next to my nose and smelling it after recording. :DIf you needed a pop filter, this will work just as well as the expensive ones, and it may even come with a pleasing aroma like mine did!Buy this product! :]",
       'The primary job of this device is to block the breath that would otherwise produce a popping sound, while allowing your voice to pass through with no noticeab

In [0]:
import pandas as pd
df = pd.read_csv("TestAPI.csv")

In [0]:
df.head(5)

,Unnamed: 0,asin,helpful,overall,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime,ReviewFeature
0,0,7806397051,"[3, 4]",1,Very oily and creamy. Not at all what I expect...,"01 30, 2014",A1YJEY40YUW4SE,Andrea,Don't waste your money,1391040000,"{'RakeTopics': ['looked awful !!!', 'expected ..."
1,1,7806397051,"[1, 1]",3,This palette was a decent price and I was look...,"04 18, 2014",A60XNB876KYML,Jessica H.,OK Palette!,1397779200,"{'RakeTopics': ['palette conceals decently', '..."
2,2,7806397051,"[0, 1]",4,The texture of this concealer pallet is fantas...,"09 6, 2013",A3G6XNM240RMWA,Karen,great quality,1378425600,"{'RakeTopics': ['professional makeup artists',..."
3,3,7806397051,"[2, 2]",2,I really can't tell what exactly this thing is...,"12 8, 2013",A1PQFP6SAJ6D80,Norah,Do not work on my face,1386460800,"{'RakeTopics': ['ish pasty fluid', 'doesnt rea..."
4,4,7806397051,"[0, 0]",3,"It was a little smaller than I expected, but t...","10 19, 2013",A38FVHZTNQ271F,Nova Amor,It's okay.,1382140800,"{'RakeTopics': ['within 30', 'long time', 'lit..."


In [0]:
import ast


c = ast.literal_eval(df['ReviewFeature'][0])

SyntaxError: ignored

In [0]:
df['ReviewFeature'][0]

['(',
 '[',
 "'",
 'l',
 'o',
 'o',
 'k',
 'e',
 'd',
 ' ',
 'a',
 'w',
 'f',
 'u',
 'l',
 ' ',
 '!',
 '!',
 '!',
 "'",
 ',',
 ' ',
 "'",
 'e',
 'x',
 'p',
 'e',
 'c',
 't',
 'e',
 'd',
 ' ',
 '.',
 '.',
 '.',
 ' ',
 'o',
 'r',
 'd',
 'e',
 'r',
 'e',
 'd',
 "'",
 ',',
 ' ',
 "'",
 't',
 'o',
 'o',
 'k',
 ' ',
 'f',
 'o',
 'r',
 'e',
 'v',
 'e',
 'r',
 "'",
 ']',
 ',',
 ' ',
 '[',
 ']',
 ',',
 ' ',
 '-',
 '0',
 '.',
 '3',
 ',',
 ' ',
 '[',
 '[',
 "'",
 'a',
 'l',
 'l',
 "'",
 ',',
 ' ',
 "'",
 'a',
 'd',
 'v',
 'm',
 'o',
 'd',
 "'",
 ',',
 ' ',
 "'",
 'o',
 'r',
 'd',
 'e',
 'r',
 'e',
 'd',
 "'",
 ',',
 ' ',
 "'",
 'V',
 'E',
 'R',
 'B',
 "'",
 ',',
 ' ',
 '[',
 'N',
 'o',
 't',
 ',',
 ' ',
 'a',
 't',
 ',',
 ' ',
 'e',
 'x',
 'p',
 'e',
 'c',
 't',
 'e',
 'd',
 ']',
 ']',
 ']',
 ',',
 ' ',
 '[',
 ']',
 ',',
 ' ',
 '[',
 '(',
 "'",
 'a',
 'w',
 'f',
 'u',
 'l',
 "'",
 ',',
 ' ',
 '0',
 '.',
 '3',
 '4',
 '8',
 '1',
 '8',
 '0',
 '2',
 '9',
 '4',
 '0',
 '2',
 '5',
 '9',
 '3',
 '5',
 '7'